In [1]:
from find_fuzzy_duplicates import find_fuzzy_duplicates
from hentOgVaskData import hent_og_vask_data

import polars as pl
pl.Config.set_tbl_rows(-1)

polars.config.Config

###### Interne dubletter
# Eksterne dubletter med Oslo/Hamar

In [2]:
###### leser inn Data exportert i fra Bergens Access database
# I fra den tabellen som heter "tblInnmeldte"
df_bergen = hent_og_vask_data()

Could not determine dtype for column 43, falling back to string


In [3]:
#df_bergen.sort(["FORNAVN", "EFTERNAVN"])

# Hamar registeret tilpasses Oslo uttrekk

In [4]:
df_convTable = df_bergen.select(
    # Columns that are renamed (or kept with the same name)
    pl.lit(0).cast(pl.Int64).alias("Cornerstone-ID"),
    pl.col("MedlemID"),  # Alias is optional when name is the same
    pl.col("Personnummer").alias("Fødselsnummer"),
    pl.col("FORNAVN").alias("Fornavn"),
    pl.lit(None).alias("Mellomnavn"),
    pl.col("EFTERNAVN").alias("Etternavn"),
    pl.col("ADRESSE").alias("Adresse"),
    pl.col("POSTNR").alias("Postnummer"),
    pl.lit(None).cast(pl.String).alias("Gruppens navn 1"),
    pl.col("Kategori").alias("Medlemstype 1"),
    pl.col("Innmeldingsdato").dt.strftime("%d.%m.%Y").alias("Innmeldt dato 1"),
    pl.col("UtmeldtDato").dt.strftime("%d.%m.%Y").alias("Utmeldt dato 1"),
    pl.col("Telefon mob").alias("Telefonnummer 1"),
    pl.col("E-postadresse").alias("E-post"),
    pl.col("RegDato").dt.strftime("%d.%m.%Y").alias("Opprettet"),
)


In [5]:
df_convTable.schema

Schema([('Cornerstone-ID', Int64),
        ('MedlemID', Int64),
        ('Fødselsnummer', String),
        ('Fornavn', String),
        ('Mellomnavn', Null),
        ('Etternavn', String),
        ('Adresse', String),
        ('Postnummer', String),
        ('Gruppens navn 1', String),
        ('Medlemstype 1', String),
        ('Innmeldt dato 1', String),
        ('Utmeldt dato 1', String),
        ('Telefonnummer 1', String),
        ('E-post', String),
        ('Opprettet', String)])

In [6]:
df_oslo = (pl.read_excel("oslo_hamar.xlsx")
)


df_oslo = df_oslo.with_columns(
    pl.when(pl.col("Mellomnavn").is_not_null())  # The condition: IF Mellomnavn is NOT null
    .then(pl.col("Fornavn") + " " + pl.col("Mellomnavn")) # THEN: concatenate the names
    .otherwise(pl.col("Fornavn")) # OTHERWISE: keep the original Fornavn
    .alias("Fornavn") # Crucial: This overwrites the existing "Fornavn" column
)

In [7]:
print(df_oslo.schema)
print(df_convTable.schema)
df_combined = pl.concat([df_oslo, df_convTable])

Schema([('Cornerstone-ID', Int64), ('MedlemID', Int64), ('Fødselsnummer', String), ('Fornavn', String), ('Mellomnavn', String), ('Etternavn', String), ('Adresse', String), ('Postnummer', String), ('Gruppens navn 1', String), ('Medlemstype 1', String), ('Innmeldt dato 1', String), ('Utmeldt dato 1', String), ('Telefonnummer 1', String), ('E-post', String), ('Opprettet', String)])
Schema([('Cornerstone-ID', Int64), ('MedlemID', Int64), ('Fødselsnummer', String), ('Fornavn', String), ('Mellomnavn', Null), ('Etternavn', String), ('Adresse', String), ('Postnummer', String), ('Gruppens navn 1', String), ('Medlemstype 1', String), ('Innmeldt dato 1', String), ('Utmeldt dato 1', String), ('Telefonnummer 1', String), ('E-post', String), ('Opprettet', String)])


In [8]:
df_combined_duplicates = find_fuzzy_duplicates(
    df=df_combined,
    columns=["Fornavn", "Etternavn"],
    columns_to_view=["Mellomnavn","Cornerstone-ID", "MedlemID", "Medlemstype 1" ,"Adresse", "Postnummer", "Innmeldt dato 1","Utmeldt dato 1", "Fødselsnummer"],
    #columns_to_view=None,
    similarity_threshold=90,
    #blocking_column="Postnummer"
)
#duplicates_found.select("fuzzy_group","MedlemID", "FORNAVN", "EFTERNAVN", "ADRESSE", "POSTNR")
df_combined_duplicates

Step 1/3: Creating candidate pairs via cross join... (this can be very slow)
 -> Found 7,728,346 pairs to compare.


Step 2/3: Calculating Similarities:   0%|          | 0/7728346 [00:00<?, ?it/s]

Step 3/3: Found 128 duplicate pairs. Building graph and assigning group IDs...


 -> Processing groups:   0%|          | 0/3809 [00:00<?, ?it/s]

fuzzy_group,Fornavn,Etternavn,Mellomnavn,Cornerstone-ID,MedlemID,Medlemstype 1,Adresse,Postnummer,Innmeldt dato 1,Utmeldt dato 1,Fødselsnummer
i64,str,str,str,i64,i64,str,str,str,str,str,str
0,"""Janicke""","""Akerø""",null,2057856,0,null,"""Svend Foyns g 69A""","""4016""",null,null,"""07115936284"""
0,"""Janicke""","""Akerø""",null,0,15536,"""M""","""Richard Nordraaks gate 9""","""5018""",null,null,"""07115936284"""
1,"""Julie Kongelstad""","""Alme""",null,2057571,0,"""Medlem Barn""","""Egnehjemveien 43""","""0196""","""01.01.2019""",null,"""02070095656"""
1,"""Mina Kongelstad""","""Alme""",null,2057572,0,"""Medlem Barn""","""Egnehjemveien 43""","""0196""","""01.01.2019""",null,"""02070095818"""
2,"""Gry Veronica""","""Alsos""",null,4681949,0,null,"""Solåsen 33B""","""5223""",null,null,"""22077028874"""
2,"""Gry Veronica""","""Alsos""",null,0,14800,"""M""","""Solåsen 33B""","""5223""","""07.10.2006""",null,"""22077028874"""
3,"""Frode Stub""","""Andersen""",null,2056303,0,null,"""Storgt. 1""","""3484""",null,null,null
3,"""Frode""","""Andersson""",null,2055759,0,"""Medlem""","""Vestfjordveien 36""","""3142""","""01.01.2019""",null,"""08035443186"""
4,"""Miriam Elisabeth Rask""","""Arnesen""",null,2056109,0,"""Støttemedlem""","""Langfjell Vestre""","""1454""","""01.01.2019""",null,"""26128746454"""


In [9]:
#df_combined_duplicates.write_excel("MedlemsDubletterOsloBergen.xlsx")

In [10]:
df_slettes = (pl.read_excel("slettes.xlsx")
)
df_slettes

MedlemID
i64
1577
4598
4593
3695
3649
4562
4516
4553
2163


In [14]:
df_combined_duplicates_bergen =(
    df_combined_duplicates.join(df_bergen,
                    on="MedlemID",
                    how="inner"
    )
)
df_combined_duplicates_bergen

fuzzy_group,Fornavn,Etternavn,Mellomnavn,Cornerstone-ID,MedlemID,Medlemstype 1,Adresse,Postnummer,Innmeldt dato 1,Utmeldt dato 1,Fødselsnummer,Innmeldingsdato,Er den innmeldte døpt ?,Menighet,Fødested,Dåpsdato,Ant barn under 15 år,Dåpskirke/sted,Fødselsdato,Personnummer,EFTERNAVN,FORNAVN,Stilling,ADRESSE,E-postadresse,POSTNR,Og,SAM_FOR,SAM_EFTER,Utsendelse,Utsendelses kode,Telefon priv,Telefon mob,Sivilstand,Trossamfunn,Gift?,Utsendelsesformat,Ektemakens trossamfunn,Anmerkning,Kategori,Giro,RegDato,SamID,FarID,MorID,Kjønn,Svart?,Faddere,Er mor og far gift?,Prest,Dåpsattest,Søk,UtmeldtDato,DødsDato,Kommune,KID-nr
i64,str,str,str,i64,i64,str,str,str,str,str,str,date,bool,str,str,date,i64,str,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,str,str,str,bool,datetime[ms],i64,i64,i64,str,bool,str,bool,str,bool,str,date,str,str,str
0,"""Janicke""","""Akerø""",null,0,15536,"""M""","""Richard Nordraaks gate 9""","""5018""",null,null,"""07115936284""",null,false,"""""","""""",null,null,"""""",1959-11-07,"""07115936284""","""Akerø""","""Janicke""","""""","""Richard Nordraaks gate 9""","""""","""5018""","""""","""""","""""","""UB""","""B""","""""","""""","""""","""Kristensamfunnet""",false,"""E""","""""","""""","""M""",false,2020-03-20 17:16:23,null,null,13033,"""""",false,"""""",false,"""""",false,"""""",null,null,"""""","""10088100739"""
2,"""Gry Veronica""","""Alsos""",null,0,14800,"""M""","""Solåsen 33B""","""5223""","""07.10.2006""",null,"""22077028874""",2006-10-07,false,"""Bergen""","""""",null,null,"""""",1970-07-22,"""22077028874""","""Alsos""","""Gry Veronica""","""Lærer""","""Solåsen 33B""","""""","""5223""","""""","""""","""""","""UB""","""BS""","""55 10 06 48""","""""","""""","""Kristensamfunnet""",false,"""E""","""Kristensamfunnet""","""""","""M""",false,2020-03-20 17:16:57,12729,null,null,"""M""",false,"""""",false,"""""",false,"""""",null,null,"""""","""10171100471"""
5,"""Bodil""","""Aukrust""",null,0,12181,"""MS""","""Haukedalsbotn 39""","""5113""","""22.12.2000""",null,"""04084728247""",2000-12-22,false,"""""","""""",null,null,"""""",1947-08-04,"""04084728247""","""Aukrust""","""Bodil""","""""","""Haukedalsbotn 39""","""""","""5113""","""og""","""Erik Øystein""","""Hauglin""","""UB""","""B""","""55 18 28 52""","""""","""""","""""",false,"""P""","""""","""""","""MS""",true,2022-02-04 16:44:28,null,null,null,"""K""",false,"""""",false,"""""",false,"""""",null,null,"""BERGEN""",""""""
6,"""Milla Kristine""","""Aukrust""",null,0,11780,"""V""","""Haukedalsbotn 39""","""5113""",null,null,null,null,false,"""""","""""",null,null,"""""",1982-12-11,null,"""Aukrust""","""Milla Kristine""","""""","""Haukedalsbotn 39""","""""","""5113""","""""","""""","""""","""""","""""","""""","""""","""""","""""",false,"""""","""""","""Bodil Aukrust, Erik Øystein Ha…","""V""",true,2000-11-11 22:54:18,null,null,12181,"""K""",false,"""""",false,"""""",false,"""""",null,null,"""BERGEN""",""""""
9,"""Erling""","""Bakken""",null,0,15610,"""V""","""Sanddalsvegen 11 D""","""5221""",null,null,"""""",null,false,"""""","""""",null,null,"""""",null,"""""","""Bakken""","""Erling""","""""","""Sanddalsvegen 11 D""","""""","""5221""","""""","""""","""""","""""","""""","""""","""""","""""","""""",false,"""""","""""","""""","""V""",false,2019-12-11 10:18:51,null,null,null,"""""",false,"""""",false,"""""",false,"""""",null,null,"""""",""""""
10,"""Jorun""","""Barane""",null,0,15537,"""M""","""Nedstagata 8""","""5745""",null,null,"""22086139603""",null,false,"""""","""""",null,null,"""""",1961-08-22,"""22086139603""","""Barane""","""Jorun""","""""","""Nedstagata 8""","""""","""5745""","""og""","""Morten""","""Clemetsen""","""""","""""","""""","""""","""""","""Kristensamfunnet""",false,"""""","""""","""""","""M""",false,2014-11-11 23:49:37,14463,null,null,"""""",false,"""""",false,"""""",false,"""""",null,null,"""""",""""""
11,"""John L.""","""Baum""",null,0,14695,"""P""","""Munkerudveien 81E""","""1165""",null,null,"""""",null,false,""

In [11]:
df_slettes_full = (
    df_slettes.join(df_bergen,
                    on="MedlemID",
                    how="left"
    )
)
df_slettes_full

MedlemID,Innmeldingsdato,Er den innmeldte døpt ?,Menighet,Fødested,Dåpsdato,Ant barn under 15 år,Dåpskirke/sted,Fødselsdato,Personnummer,EFTERNAVN,FORNAVN,Stilling,ADRESSE,E-postadresse,POSTNR,Og,SAM_FOR,SAM_EFTER,Utsendelse,Utsendelses kode,Telefon priv,Telefon mob,Sivilstand,Trossamfunn,Gift?,Utsendelsesformat,Ektemakens trossamfunn,Anmerkning,Kategori,Giro,RegDato,SamID,FarID,MorID,Kjønn,Svart?,Faddere,Er mor og far gift?,Prest,Dåpsattest,Søk,UtmeldtDato,DødsDato,Kommune,KID-nr
i64,date,bool,str,str,date,i64,str,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,str,str,str,bool,datetime[ms],i64,i64,i64,str,bool,str,bool,str,bool,str,date,str,str,str
1577,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
4598,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
4593,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3695,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3649,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
4562,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
4516,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
4553,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2163,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [12]:
#df_slettes_full.write_excel("DubletterOsloBergen.xlsx")